## Let's try Single Block Encoding Approach

For instance, we convert ATATGATCGA to 101001001, 110100,  101, 1 then to 2331, 123, 12, 1?

In [42]:
from sbe import SBE_f,SBE_b,SBE_convert_f,SBE_convert_b

temp = SBE_convert_f('ATCATASFEGATCG')
SBE_convert_b(temp)

temp = SBE_f([15,16,15,14,14,14,15,16,15,14,14,14])
print(temp)
SBE_b(temp)

[15, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 16, 1, 0, 0, 0, 1, 0, 0, 0, 14, 1, 1, 1, 1, 1, 1]


[15, 16, 15, 14, 14, 14, 15, 16, 15, 14, 14, 14]

In [38]:
import arithc as arith
import fqt, ppm
import contextlib, sys
import filecmp
from IPython.display import clear_output

In [39]:
def SBEcompress(inp, bitout):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs)
    enc = arith.ArithmeticCoder(32)
    enc.start_encode(bitout) # New line!
    s = []
    while True:
        symbol = inp.read(1)
        if len(symbol) == 0:
                break
        s += [symbol[0]+2] ## leave room for 0,1
    sbe = SBE_f(s)
    sbe_list = sbe
    print(len(sbe))
    idx = 0
    # prior
    model.set(0,1000000)
    model.set(1,1000000)
    while True:
        # Read and encode one byte
        if idx == len(sbe):
            break
        symbol = sbe_list[idx]
        idx += 1
        if idx % 1100000 == 0:
            print(str(10*idx//(len(sbe)//10)) + ' percent done')
            clear_output(wait = True)
        t = model.get_total() ## New lines!
        l = model.get_low(symbol)
        h = model.get_high(symbol)
        enc.storeRegion(l,h,t) 
        model.increment(symbol)
    t = model.get_total() ## New lines!
    l = model.get_low(256)
    h = model.get_high(256)
    enc.storeRegion(l,h,t)
    enc.finish_encode(bitout)  # New line!
inputfile, outputfile = 'data\ecoli\Ecoli.txt', 'data\ecoli\Ecoli_sbe_prior.txt'

#Perform file compression
with open(inputfile, "rb") as inp, \
        contextlib.closing(arith.BitOutputStream(open(outputfile, "wb"))) as bitout:
    SBEcompress(inp, bitout)

94 percent done


In [44]:
def decompress(bitin, out):
    initfreqs = fqt.FlatFrequencyTable(257)
    model = fqt.SimpleFrequencyTable(initfreqs)
    dec = arith.ArithmeticCoder(32)
    dec.start_decode(bitin) # New line!
    sbe = []
    while True:
        # Decode and write one byte
        total = model.get_total()
        Range = dec.R
        offset = dec.getTarget()#dec.D - dec.L
        #value_0 = ((offset + 1) * total - 1)// Range
        value = dec.getTarget(total)

        start = 0
        end = model.get_symbol_limit()
        while end - start > 1:
            middle = (start + end) >> 1
            if model.get_low(middle) > value:
                end = middle
            else:
                start = middle

        symbol = start

        #self 
        l = model.get_low(symbol) 
        h = model.get_high(symbol)
        dec.loadRegion(l,h,total)
        #print(symbol)
        model.increment(symbol)
        if symbol == 256:  # EOF symbol
            break
        sbe += [symbol]
    print(len(sbe))
    s_temp = SBE_b(sbe)
    s = [x - 2 for x in s_temp]
    print(len(s))
    for symbol in s:
        out.write(bytes((symbol,)))
    return sbe, s

        
inputfile, outputfile = 'data\ecoli\Fake_Ecoli_sbe_compressed.txt', 'data\ecoli\Fake_Ecoli_sbe_decompressed.txt'

# Perform file decompression
with open(inputfile, "rb") as inp, open(outputfile, "wb") as out:
    bitin = arith.BitInputStream(inp)
    sbe, s = decompress(bitin, out)


filecmp.cmp('data\ecoli\Fake_Ecoli.txt', 'data\ecoli\Fake_Ecoli_sbe_decompressed.txt')

11671499
4638690


True